In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, classification_report, roc_auc_score, mean_absolute_error, average_precision_score
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
from lifelines import CoxPHFitter

## Prepare Data
Separate out the outcomes. Keep only mean and last BMI. 

Split into train and test datasets.

Standardize the lab values, age at first diagnosis, and BMI (train the scaler on the training set and then use it on the test set).

In [ ]:
df = pd.read_csv("/nobackup/users/ericason/mlhc-final-project/clean_data/nafl/combined.large.nafl.csv", header=0, delimiter=",")
df.head()

In [ ]:
# make lists of important columns
outcome_cols = ["Outcome", "DaysUntilFirstProgression"] # outcomes
drop_cols = ["StudyID"] # columns to drop that are not outcome
# columns that should be scaled later
numerical_cols = [x for x in df.columns if ("lab" in x.lower()) or ("age" in x.lower()) or ("bmi" in x.lower() and "category" not in x.lower())]


In [ ]:
# make features dataframe
X = df.drop(columns=outcome_cols + drop_cols)
X.head()

In [ ]:
# make outcome dataframe (including both linear and logistic outcomes)
Y = df[["DaysUntilFirstProgression", "Outcome"]]

In [ ]:
# make train test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)
print(f'X_train shape: {X_train.shape}, X_test shape: {X_test.shape}, Y_train shape: {Y_train.shape}, Y_test shape: {Y_test.shape}')

In [ ]:
# standardize numerical columns
scaler = StandardScaler()
X_train_scaled = X_train
# scale numerical columns and replace them in the original dataframe
X_train_scaled[numerical_cols] = scaler.fit_transform(X_train[numerical_cols]) 
X_train_scaled.head()

In [ ]:
X_test_scaled = X_test
X_test_scaled[numerical_cols] = scaler.transform(X_test[numerical_cols])

## Fit Linear Regression
Fit a linear regression on DaysUntilFirstProgression

Rank features by coefficients (both most positive and most negative)

Check VIF scores

In [ ]:
# fit the model
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, Y_train['DaysUntilFirstProgression'])

In [ ]:
prediction = linear_model.predict(X_test_scaled)
print(prediction)

In [ ]:
train_mse_loss = mean_squared_error(Y_train['DaysUntilFirstProgression'], linear_model.predict(X_train_scaled))
test_mse_loss = mean_squared_error(Y_test['DaysUntilFirstProgression'], prediction)
print("Train MSE Loss: ", train_mse_loss)
print("Test MSE Loss: ", test_mse_loss)

In [ ]:
print("Train MAE Loss: ", mean_absolute_error(Y_train['DaysUntilFirstProgression'], linear_model.predict(X_train_scaled)))
print("Train MAE Loss: ", mean_absolute_error(Y_test['DaysUntilFirstProgression'], prediction))

In [ ]:
# rank by coefficient values
sorted_coefs = X_train_scaled.columns[np.argsort(linear_model.coef_)]
most_positive = sorted_coefs[-10:]
most_negative = sorted_coefs[:10]

In [ ]:
print(most_negative) # most negatively correlated with outcome (duration)

In [ ]:
print(most_positive) # most positively correlated with outcome (duration)

In [ ]:
med_df = pd.read_csv("/nobackup/users/ericason/mlhc-final-project/data/NAFLpatients_Jan2025request/Med_all.use.final.txt", delimiter="\t", header=0)
med_df.head()

In [ ]:
med_codes = "MedType_Code_" + med_df["Code_Type"] + "_" + med_df["Code"]

In [ ]:
med_codes_df = pd.concat([med_codes, med_df["Medication"]], axis=1)
med_codes_df.columns = ["Code", "Medication"]
med_codes_df.head()

In [ ]:
med_codes_df = med_codes_df.drop_duplicates() # drop duplicate codes and medications
med_codes_df.shape

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_positive)] # most positive descriptions

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_negative)] # note that there are only 9 rows because one of the features was unknown sex

In [ ]:
# get coefficients that are closest to 0
abs_sorted_coefs = X_train_scaled.columns[np.argsort(abs(linear_model.coef_))][:10]
print(abs_sorted_coefs)

## Fit Logistic Regression
Fit a logistic regression on outcome, check VIF, rank features by coefficients

In [ ]:
logistic_model = LogisticRegression()
logistic_model.fit(X_train_scaled, Y_train['Outcome']) # train model

In [ ]:
# make predictions on training and test data
train_logistic_predictions = logistic_model.predict(X_train_scaled)
test_logistic_predictions = logistic_model.predict(X_test_scaled)

In [ ]:
# check classification report for training
print(classification_report(Y_train['Outcome'], train_logistic_predictions, target_names=['No Progression', 'Progression']))

In [ ]:
print(confusion_matrix(Y_train['Outcome'], train_logistic_predictions))

In [ ]:
print("Training AUROC: ", roc_auc_score(Y_train['Outcome'], train_logistic_predictions))

In [ ]:
print("Training AUPRC: ", average_precision_score(Y_train['Outcome'], train_logistic_predictions))

In [ ]:
# check classification report for testing
print(classification_report(Y_test['Outcome'], test_logistic_predictions, target_names=['No Progression', 'Progression']))

In [ ]:
print(confusion_matrix(Y_test['Outcome'], test_logistic_predictions))

In [ ]:
print("Testing AUROC: ", roc_auc_score(Y_test['Outcome'], test_logistic_predictions))

In [ ]:
print("Testing AUPRC: ", average_precision_score(Y_test['Outcome'], test_logistic_predictions))

In [ ]:
# rank by coefficient values
sorted_coefs = X_train_scaled.columns[np.argsort(logistic_model.coef_)[0]]
most_positive = sorted_coefs[-10:]
most_negative = sorted_coefs[:10]

In [ ]:
print(most_negative) # most negatively correlated with outcome (progression binary class)

In [ ]:
print(logistic_model.coef_[0][np.argsort(logistic_model.coef_)[0]][:10])

In [ ]:
print(most_positive) # most positively correlated with outcome (progression binary class)

In [ ]:
print(logistic_model.coef_[0][np.argsort(logistic_model.coef_)[0]][-10:])

In [ ]:
abs_sorted_coefs = X_train_scaled.columns[np.argsort(abs(logistic_model.coef_))[0]]
abs_sorted_coefs[:10] # codes with coefficients closest to zero

In [ ]:
print(logistic_model.coef_[0][np.argsort(abs(logistic_model.coef_))[0]][:10])

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_positive)] # most positive descriptions

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_negative)] # most negative descriptions

In [ ]:
med_codes_df[med_codes_df['Code'].isin(abs_sorted_coefs[:10])] # descriptions for close to 0

## Fit Cox PH model
Fit a standard Cox PH model with linear proportional hazards assumption, rank features by coefficient

In [ ]:
# use lasso to reduce collinearity
from sklearn.linear_model import LassoCV

lasso = LassoCV(cv=5).fit(X_train_scaled, Y_train["DaysUntilFirstProgression"])
selected_features = X_train_scaled.columns[(lasso.coef_ != 0)]
print("Selected features:", selected_features.tolist())

In [ ]:
# load data
data = pd.concat([X_train_scaled[selected_features.to_list()], Y_train], axis=1)

In [ ]:
# Fit the Cox model
cph = CoxPHFitter()
cph.fit(data, duration_col='DaysUntilFirstProgression', event_col='Outcome')

In [ ]:
# rank by coefficient values
sorted_coefs = X_train_scaled.columns[np.argsort(cph.params_)]
most_positive = sorted_coefs[-10:]
most_negative = sorted_coefs[:10]


In [ ]:
print(most_negative) # most negatively correlated with outcome (progression binary class)

In [ ]:
print(most_positive) # most positiveely correlated with outcome (progression binary class)

In [ ]:
lab_df = pd.read_csv("/nobackup/users/ericason/mlhc-final-project/data/NAFLpatients_Jan2025request/Lab_all.use.final.txt", delimiter="\t", header=0)
lab_df.head()

In [ ]:
lab_codes = "Lab_" + lab_df["Loinc_Code"]

In [ ]:
lab_codes_df = pd.concat([lab_codes, lab_df["Test_Description"]], axis=1)
lab_codes_df.columns = ["Code", "Lab Test"]
lab_codes_df.head()

In [ ]:
lab_codes_df = lab_codes_df.drop_duplicates() # drop duplicate codes and medications
lab_codes_df.shape

In [ ]:
lab_codes_df[lab_codes_df['Code'].isin(most_negative)] # most negative labs

In [ ]:
pd.set_option('display.max_colwidth', 200)

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_negative)] # most negative meds

In [ ]:
med_codes_df[med_codes_df['Code'].isin(most_positive)] # most positive meds

In [ ]:
from lifelines.utils import concordance_index
from sklearn.metrics import brier_score_loss

In [ ]:
# Predict partial hazards (or risk scores)
risk_scores = -cph.predict_partial_hazard(X_train).values.ravel()

# Compute concordance index
c_index_train = concordance_index(event_times=Y_train["DaysUntilFirstProgression"], predicted_scores=risk_scores, event_observed=Y_train["Outcome"])

print("Test Concordance Score:", c_index_train)

In [ ]:
# Predict partial hazards (or risk scores)
risk_scores = -cph.predict_partial_hazard(X_test).values.ravel()

# Compute concordance index
c_index_test = concordance_index(event_times=Y_test["DaysUntilFirstProgression"], predicted_scores=risk_scores, event_observed=Y_test["Outcome"])

print("Test Concordance Score:", c_index_test)

In [ ]:
# Time points to evaluate
times = np.linspace(0, 2247, 100)
brier_scores = []

# Loop through time points and compute approximate Brier score
for t in times:
    surv_probs = cph.predict_survival_function(X_train, times=[t]).T[t]
    # Binary outcome: 1 if event occurred before t, else 0
    observed = ((Y_train["DaysUntilFirstProgression"] <= t) & (Y_train["Outcome"] == 1)).astype(int)
    # Approximate Brier score
    brier = brier_score_loss(observed, 1 - surv_probs)
    brier_scores.append(brier)

integrated_brier_score = np.trapz(brier_scores, times) / (times[-1] - times[0])
print("Test integrated brier score: ", integrated_brier_score)

In [ ]:
# Time points to evaluate
times = np.linspace(0, 2247, 100)
brier_scores = []

# Loop through time points and compute approximate Brier score
for t in times:
    surv_probs = cph.predict_survival_function(X_test, times=[t]).T[t]
    # Binary outcome: 1 if event occurred before t, else 0
    observed = ((Y_test["DaysUntilFirstProgression"] <= t) & (Y_test["Outcome"] == 1)).astype(int)
    # Approximate Brier score
    brier = brier_score_loss(observed, 1 - surv_probs)
    brier_scores.append(brier)

# Plot
plt.plot(times, brier_scores, label="Approx. Brier Score")
plt.xlabel("Time t (days)")
plt.ylabel("Brier Score")
plt.ylim(0, 0.9)
plt.title("Cox PH Regression Brier Score Over Time")
plt.tight_layout()
plt.savefig("./brier.png")
plt.show()

In [ ]:
integrated_brier_score = np.trapz(brier_scores, times) / (times[-1] - times[0])
print("Test integrated brier score: ", integrated_brier_score)

## Get Human Readable Content

In [ ]:
cox_codes = ['Lab_1920-8',
'Lab_2500-7',
'Code_R07.9',
'Code_I10',
'Lab_718-7',
'Lab_2091-7',
'Code_E11.9',
'Lab_785-6',
'Code_R94.5',
'Lab_62292-8',
'Lab_4544-3',
'Lab_787-2',
'Code_D89.89',
'last_BMI',
'Lab_777-3',
'Lab_9830-1',
'Lab_789-8',
'Code_E55.9',
'Code_R91.8',
'Code_Z71.89']

In [ ]:
lab_codes_df[lab_codes_df['Code'].isin(cox_codes)] # most positive meds

In [ ]:
dia_df = pd.read_csv("/nobackup/users/ericason/mlhc-final-project/data/NAFLpatients_Jan2025request/Dia_all.use.final.txt", delimiter="\t", header=0)

In [ ]:
dia_df.head()

In [ ]:
dia_codes = "Code_" + dia_df["Code"]

In [ ]:
dia_codes_df = pd.concat([dia_codes, dia_df["Diagnosis_Name"]], axis=1)
dia_codes_df.columns = ["Code", "Diagnosis"]
dia_codes_df.head()

In [ ]:
dia_codes_df = dia_codes_df.drop_duplicates() # drop duplicate codes and medications
dia_codes_df.shape

In [ ]:
dia_codes_df[dia_codes_df['Code'].isin(cox_codes)] # most positive meds

In [ ]:
linear_nn_codes = ['Lab_4679-7',
'Lab_14338-8',
'Lab_2132-9',
'Lab_6768-6',
'Code_Z23',
'Lab_6690-2',
'Lab_2093-3',
'MedType_Code_EPIC-MED_10328',
'Lab_13457-7',
'Lab_2571-8']

In [ ]:
lab_codes_df[lab_codes_df['Code'].isin(linear_nn_codes)] 

In [ ]:
med_codes_df[med_codes_df['Code'].isin(linear_nn_codes)] 

In [ ]:
dia_codes_df[dia_codes_df['Code'].isin(linear_nn_codes)] 

In [ ]:
linear2_codes = ['MedType_Code_EPIC-MED_27698',
'Lab_19153-6',
'Lab_2501-5',
'Lab_786-4',
'Code_E78.5',
'Lab_XC5-9',
'Lab_2502-3',
'MedType_Code_EPIC-MED_40900',
'Lab_2089-1',
'Lab_789-8']

In [ ]:
lab_codes_df[lab_codes_df['Code'].isin(linear2_codes)] 

In [ ]:
dia_codes_df[dia_codes_df['Code'].isin(linear2_codes)] 

In [ ]:
med_codes_df[med_codes_df['Code'].isin(linear2_codes)] 